In [1]:
import os
import pickle
import time
import pandas as pd
import yfinance as yf
from fredapi import Fred
#from alpha_vantage.fundamentaldata import FundamentalData
from dotenv import load_dotenv
import pandas_ta as ta

load_dotenv()

# Set API keys
#ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
FRED_API_KEY = os.getenv('FRED_API_KEY')

#if not ALPHA_VANTAGE_API_KEY:
#    raise ValueError("Set Alpha Vantage API key in the '.env' file.")
if not FRED_API_KEY:
    raise ValueError("Set FRED API key in the '.env' file.")

fred = Fred(api_key=FRED_API_KEY)
#alpha_fd = FundamentalData(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')


In [2]:
START_DATE = '2017-01-01'
END_DATE = '2023-12-31'


COMPANIES = {
    'Telus': ['T.TO', 'TU'],
    'Rogers Communications': ['RCI-B.TO', 'RCI'],
    'Quebecor': ['QBR-B.TO'],
    'Cogeco Communications': ['CCA.TO'],
    'Bell Canada (BCE Inc.)': ['BCE.TO', 'BCE'],
    'Hydro One': ['H.TO'],
    'Fortis': ['FTS.TO', 'FTS'],
    'AltaGas': ['ALA.TO'],
    'Canadian National Railway': ['CNR.TO', 'CNI'],
    'Canadian Pacific Railway': ['CP.TO', 'CP'],
    'Metro': ['MRU.TO'],
    'Loblaws': ['L.TO'],
    'Empire': ['EMP-A.TO'],
    'Alimentation Couche-Tard': ['ATD-B.TO', 'ATD'],
}


MACRO_INDICATORS = {
    'GDP': {'series_id': 'GDP', 'frequency': 'Quarterly'},
    'CPI': {'series_id': 'CPIAUCSL', 'frequency': 'Monthly'},
    'Unemployment Rate': {'series_id': 'UNRATE', 'frequency': 'Monthly'},
    'Interest Rate': {'series_id': 'FEDFUNDS', 'frequency': 'Monthly'},
    'Industrial Production': {'series_id': 'INDPRO', 'frequency': 'Monthly'},
    'Consumer Confidence': {'series_id': 'UMCSENT', 'frequency': 'Monthly'},
    'Retail Sales': {'series_id': 'RSAFS', 'frequency': 'Monthly'},
    'Housing Starts': {'series_id': 'HOUST', 'frequency': 'Monthly'},
    'Durable Goods Orders': {'series_id': 'DGORDER', 'frequency': 'Monthly'},
    'Producer Price Index': {'series_id': 'PPIACO', 'frequency': 'Monthly'},
}

In [ ]:
#def fetch_data_with_retry(fetch_function, *args, max_retries=3, sleep_time=15, **kwargs):
#    for attempt in range(max_retries):
#        try:
#            return fetch_function(*args, **kwargs)
#        except Exception as e:
#            print(f"Error: {e}. Retrying in {sleep_time} seconds...")
#            time.sleep(sleep_time)
#    print("Max retries exceeded.")
#    return None

In [ ]:
def get_technical_data(companies, start_date, end_date):
    technical_data_list = []
    for company, tickers in companies.items():
        data_fetched = False
        for ticker in tickers:
            try:
                data = yf.download(ticker, start=start_date, end=end_date)
                if data.empty:
                    continue
                data.reset_index(inplace=True)
                data.columns = data.columns.get_level_values(0)
                # Calculate returns
                data['Daily Return'] = data['Close'].pct_change()
                data['Adjusted Daily Return'] = data['Adj Close'].pct_change()
                # Add company and ticker info
                data['Company'] = company
                data['Ticker'] = ticker
                # Calculate technical indicators
                data = calculate_technical_indicators(data)
                technical_data_list.append(data)
                print(f"Technical data fetched for {company} ({ticker})")
                data_fetched = True
                break
            except Exception as e:
                print(f"Error fetching technical data for {company} ({ticker}): {e}")
        if not data_fetched:
            print(f"Failed to fetch technical data for {company}")
    # Combine all data into one DataFrame
    technical_data = pd.concat(technical_data_list, ignore_index=True)
    return technical_data

def calculate_technical_indicators(data):
    # Moving Averages
    data['SMA_20'] = ta.sma(data['Close'], length=20)
    data['EMA_20'] = ta.ema(data['Close'], length=20)
    # Bollinger Bands
    bbands = ta.bbands(data['Close'], length=20)
    data = pd.concat([data, bbands], axis=1)
    # Relative Strength Index
    data['RSI_14'] = ta.rsi(data['Close'], length=14)
    # Moving Average Convergence Divergence
    macd = ta.macd(data['Close'])
    data = pd.concat([data, macd], axis=1)
    return data

def melt_technical_data(data):
    """
    Melt the technical data DataFrame into long format.
    """
    id_vars = ['Company', 'Ticker', 'Date']
    value_vars = [col for col in data.columns if col not in id_vars]
    melted_data = data.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='Variable',
        value_name='Value'
    )
    return melted_data

In [ ]:
technical_data = get_technical_data(COMPANIES, START_DATE, END_DATE)
technical_data['Date'] = pd.to_datetime(technical_data['Date']).dt.date
technical_data = technical_data[technical_data['Date'] >= pd.to_datetime('2018-01-01').date()]
technical_melted = melt_technical_data(technical_data)

In [ ]:
def get_macro_data(indicators, start_date, end_date):
    macro_data_list = []
    for indicator_name, info in indicators.items():
        series_id = info['series_id']
        frequency = info['frequency']
        try:
            data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
            data = data.to_frame(name='Value')
            data['Indicator'] = indicator_name
            data['Frequency'] = frequency
            data.index.rename('Date', inplace=True)
            data.reset_index(inplace=True)

            # Calculate one-period percentage change
            data['Percent_Change'] = data['Value'].pct_change()

            # Create a separate DataFrame for the percentage change
            percent_change_data = data[['Date', 'Percent_Change', 'Indicator', 'Frequency']].copy()
            percent_change_data.rename(columns={'Percent_Change': 'Value'}, inplace=True)
            percent_change_data['Indicator'] = percent_change_data['Indicator'] + " Pct Change"

            macro_data_list.append(data)
            macro_data_list.append(percent_change_data)
            
            print(f"Macro data fetched for {indicator_name}")
        except Exception as e:
            print(f"Error fetching macro data for {indicator_name} ({series_id}): {e}")

    # Combine all data into one DataFrame
    macro_data = pd.concat(macro_data_list, ignore_index=True)
    macro_data = macro_data.drop(["Percent_Change"], axis=1)
    return macro_data

In [ ]:
macro_data = get_macro_data(MACRO_INDICATORS, START_DATE, END_DATE)

In [ ]:
# Save the outputs as pickle files
#with open('macro_data.pkl', 'wb') as macro_file:
#    pickle.dump(macro_data, macro_file)

#with open('technical_data.pkl', 'wb') as technical_file:
#    pickle.dump(technical_data, technical_file)

#with open('technical_melted.pkl', 'wb') as melted_file:
#    pickle.dump(technical_melted, melted_file)